In [116]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.model_selection import cross_val_score
from sklearn import preprocessing



In [117]:
train = pd.read_csv("Input/train.csv", index_col = 0)

In [118]:
train

,carat,cut,color,clarity,depth,table,x,y,z,price
id,,,,,,,,,,
0,0.30,Very Good,F,VS2,62.8,56.0,4.29,4.31,2.70,605
1,0.34,Ideal,E,SI1,62.6,55.0,4.46,4.49,2.80,565
2,0.40,Very Good,D,SI1,60.3,62.0,4.70,4.75,2.85,720
3,0.40,Premium,H,VS1,61.8,59.2,4.72,4.74,2.92,793
4,0.90,Very Good,D,SI1,61.0,63.0,6.10,6.13,3.73,4381
...,...,...,...,...,...,...,...,...,...,...
40450,0.32,Ideal,D,VS1,61.5,56.0,4.41,4.43,2.72,862
40451,0.30,Ideal,D,VS2,60.9,57.0,4.32,4.35,2.64,710
40452,0.47,Premium,G,SI1,62.7,58.0,4.97,4.99,3.12,931


In [74]:
#train=pd.get_dummies(train)

In [119]:
le = preprocessing.LabelEncoder()
train["cut"] = le.fit_transform(train["cut"])
train["color"] = le.fit_transform(train["color"])
train["clarity"] = le.fit_transform(train["clarity"])

In [121]:
columns = ['carat','depth', 'table', 'x', 'y', 'z']
for col in columns:
    train[col] = (train[col] - np.mean(train[col])) / np.std(train[col]) 
train.head()

,carat,cut,color,clarity,depth,table,x,y,z,price
id,,,,,,,,,,
0,-1.051355,4,2,5,0.736126,-0.650176,-1.283716,-1.236438,-1.180531,605
1,-0.966764,2,1,2,0.596363,-1.097476,-1.132102,-1.080061,-1.039596,565
2,-0.839879,4,0,2,-1.010919,2.033618,-0.918059,-0.854183,-0.969129,720
3,-0.839879,3,4,4,0.037308,0.781181,-0.900222,-0.862870,-0.870475,793
4,0.217504,4,0,2,-0.521746,2.480917,0.330526,0.344710,0.271096,4381


In [122]:
train["volume"]=train['x']*train['y']*train['z']


In [123]:
train.drop(["x","y","z"],axis = 1, inplace = True)

In [124]:
train.head()

,carat,cut,color,clarity,depth,table,price,volume
id,,,,,,,,
0,-1.051355,4,2,5,0.736126,-0.650176,605,-1.873780
1,-0.966764,2,1,2,0.596363,-1.097476,565,-1.271155
2,-0.839879,4,0,2,-1.010919,2.033618,720,-0.759981
3,-0.839879,3,4,4,0.037308,0.781181,793,-0.676163
4,0.217504,4,0,2,-0.521746,2.480917,4381,0.030888


## Training

In [125]:
X=train.drop(columns=["price"])
y=train["price"]

In [126]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(32364, 7) (8091, 7) (32364,) (8091,)


In [128]:
model= RandomForestRegressor(n_estimators=1500).fit(X_train, y_train)
y_pred=model.predict(X_test)
print(np.mean(cross_val_score(model, X_train, y_train,scoring='r2', cv=3, n_jobs=2)))
print("r2_score", r2_score(y_test,y_pred))
print("RSME", (mean_squared_error(y_test,y_pred)**0.5))

0.9781652015443828
r2_score 0.9805064665654089
RSME 564.0568665148144


In [127]:
model2= GradientBoostingRegressor(n_estimators=1000).fit(X_train, y_train)
y_pred=model2.predict(X_test)
print(np.mean(cross_val_score(model2, X_train, y_train, scoring='r2', cv=3, n_jobs=2)))
print("r2_score", r2_score(y_test,y_pred))
print("RSME", (mean_squared_error(y_test,y_pred)**0.5))

0.9779433115235098
r2_score 0.9793286052869743
RSME 580.8479978236089


## TESTING

In [129]:
test = pd.read_csv("Input/predict.csv", index_col = 0)
test.head()

,carat,cut,color,clarity,depth,table,x,y,z
id,,,,,,,,,
0,2.36,Ideal,I,SI2,60.8,54.0,8.68,8.57,5.24
1,2.04,Ideal,H,SI2,62.0,56.0,8.18,8.23,5.09
2,0.51,Ideal,I,SI1,61.7,54.0,5.18,5.19,3.20
3,0.30,Ideal,I,SI1,61.3,56.0,4.32,4.33,2.65
4,0.96,Fair,H,VS2,68.8,56.0,6.11,5.98,4.16


In [130]:
le = preprocessing.LabelEncoder()
test["cut"] = le.fit_transform(test["cut"])
test["color"] = le.fit_transform(test["color"])
test["clarity"] = le.fit_transform(test["clarity"])

In [131]:
columns = ['carat','depth', 'table', 'x', 'y', 'z']
for col in columns:
    test[col] = (test[col] - np.mean(test[col])) / np.std(test[col]) 
test.head()

,carat,cut,color,clarity,depth,table,x,y,z
id,,,,,,,,,
0,3.267015,2,5,3,-0.666318,-1.554540,2.620734,2.539800,2.446736
1,2.596725,2,4,3,0.168508,-0.658049,2.175567,2.234832,2.230593
2,-0.608098,2,5,2,-0.040199,-1.554540,-0.495431,-0.491942,-0.492811
3,-1.047976,2,5,2,-0.318474,-0.658049,-1.261117,-1.263332,-1.285335
4,0.334497,0,4,5,4.899190,-0.658049,0.332578,0.216660,0.890505


In [132]:
test["volume"]=test['x']*test['y']*test['z']

In [133]:
test.drop(["x","y","z"],axis = 1, inplace = True)

In [134]:
test.head()

,carat,cut,color,clarity,depth,table,volume
id,,,,,,,
0,3.267015,2,5,3,-0.666318,-1.554540,16.285810
1,2.596725,2,4,3,0.168508,-0.658049,10.845199
2,-0.608098,2,5,2,-0.040199,-1.554540,-0.120109
3,-1.047976,2,5,2,-0.318474,-0.658049,-2.047808
4,0.334497,0,4,5,4.899190,-0.658049,0.064167


In [135]:
price_pred3=model.predict(test)

In [136]:
df_price3=pd.DataFrame({'price':price_pred3})
df_price3.index.rename('id', inplace=True)
df_price3.head()

,price
id,
0,14055.420000
1,14528.247333
2,990.639333
3,469.348000
4,3933.438000


In [137]:
df_price3.to_csv("output/test3.csv")